# Fantasy Football Quantile Predictions

This notebook predicts **floor, median, and ceiling** fantasy points for your ESPN roster using quantile regression.

## What You'll Get
- **pred_10 (Floor)**: Safe minimum - player should score above this 90% of the time
- **pred_50 (Median)**: Expected score - most likely outcome
- **pred_90 (Ceiling)**: Upside - player could score this high 10% of the time

## Setup Instructions

1. **Install dependencies**: `pip install espn_api nfl_data_py scikit-learn`

2. **Get your ESPN credentials**:
   - `league_id`: From your league URL (e.g., `leagueId=567575`)
   - `espn_s2` and `swid`: Browser cookies from [this guide](https://github.com/cwendt94/espn-api/discussions/150)

3. **Run all cells in order** - predictions will appear at the end!

## Why Quantile Regression?

**The Problem**: Standard projections give you one number (e.g., "14.5 points"). But two players can both project 14.5 points with very different risk profiles:
- Player A: Consistent, always scores 13-16 points
- Player B: Boom/bust, scores 5-24 points

**The Solution**: Instead of one prediction, we give you THREE:
- **Floor (10th percentile)**: "There's a 90% chance they score at least this much"
- **Median (50th percentile)**: "This is the most likely outcome"
- **Ceiling (90th percentile)**: "There's a 10% chance they score this much or more"

**How it works**:
1. Load 3+ years of NFL play-by-play data
2. Calculate weekly fantasy points for each player
3. Build features for each player-week:
   - **Game stats**: yards, TDs, interceptions, EPA, CPOE
   - **Historical stats**: last week's points, 3-week rolling avg, career avg
4. Train 3 separate Gradient Boosting models (one per quantile)
5. **At prediction time**: Use ESPN's projected stats + player's historical patterns

---
## Step 1: Connect to ESPN League


In [2]:
# Connect to ESPN Fantasy League
from espn_api.football import League
import nfl_data_py as nfl

league = League(
    league_id=567575, 
    year=2025, 
    espn_s2='AECVN3FcAWfB56xxM5SnNVqsxq9soOxMmzDH1CfYYOkX3KIrzeGSsTMZ0CwJLPQoBYxLMp59ILoZ0CvUnTrBbU15b2PwD1v9fZRoO5iMb%2Fy%2FWPaOqPwlwSx2ShvBAt%2BSqJxtboHzcpSuSOgASzSNx4divXOEc4aVZjnOx7qRJ9YbE800NnLCNiLMBpaHjdZg%2BMN6vwCInJrKejPDXsmdjo%2FIkV0IfCLQHr6QHyJjLhqOwAPozqNPyGa1ZZT8DOxA%2BmpTsa5v9cgfJ4V%2BVZzxzr95KxBS0k%2BYJMt7OWSdA%2B2yUQ%3D%3D', 
    swid='{280AA84B-DE12-4B3D-80F2-283BF634242B}'
)
team = league.teams[9]  # <-- Change index to select your team
print(f"Connected to league. Your team: {team.team_name}")



Connected to league. Your team: Math Guys


---
## Step 2: Load Data & Train Model

The next few cells:
1. **Define scoring rules** (Half-PPR: 0.5 pts per reception)
2. **Load NFL play-by-play data** (2022-2025 seasons)
3. **Build features** for each player-week:
   - **Game stats**: passing/rushing/receiving yards, TDs, interceptions, EPA, CPOE
   - **Historical stats**: `Y_lag_1` (last week), `Y_roll_avg_3` (3-week avg), `Y_cum_avg` (career)
4. **Train 3 Gradient Boosting models** (one for each quantile: 10%, 50%, 90%)

*Note: At prediction time, we use ESPN's projected stats instead of actual game stats (which we don't have yet).*


In [3]:
# Helpers: fantasy scoring per play and weekly target aggregation
import pandas as pd
import numpy as np

# Half-PPR scoring rules
scoring_rules = {
    'pass_yd': 0.04, 'pass_td': 4,
    'rush_yd': 0.1, 'rush_td': 6,
    'rec_yd': 0.1, 'rec_td': 6,
    'rec': 0.5,
    'int': -2, 'fumble_lost': -2,
    'qb_kneel_yd': -0.1
}

def calculate_fantasy_points_per_play(df: pd.DataFrame, scoring_rules: dict) -> pd.DataFrame:
    df = df.copy()
    df['fp_pass'] = (df['passing_yards'].fillna(0) * scoring_rules['pass_yd']) + \
                    (df['pass_touchdown'].fillna(0) * scoring_rules['pass_td'])
    df['fp_rush'] = (df['rushing_yards'].fillna(0) * scoring_rules['rush_yd']) + \
                    (df['rush_touchdown'].fillna(0) * scoring_rules['rush_td'])
    df['fp_rec'] = (df['receiving_yards'].fillna(0) * scoring_rules['rec_yd']) + \
                   (df['pass_touchdown'].fillna(0) * scoring_rules['rec_td']) + \
                   (df['complete_pass'].fillna(0) * scoring_rules['rec'])
    df['fp_int'] = df['interception'].fillna(0) * scoring_rules['int']
    df['fp_fumble_lost'] = df['fumble_lost'].fillna(0) * scoring_rules['fumble_lost']
    df['fp_kneel'] = np.where(df['play_type'] == 'qb_kneel',
                              df['yards_gained'].fillna(0) * scoring_rules['qb_kneel_yd'], 0)
    return df

def calculate_weekly_fantasy_points_final(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['fp_pass_total'] = df['fp_pass'].fillna(0) + df['fp_int'].fillna(0) + df['fp_kneel'].fillna(0)
    df['fp_rush_total'] = df['fp_rush'].fillna(0)
    df['fp_rec_total'] = df['fp_rec'].fillna(0)
    df['fp_fumble_1_total'] = df['fp_fumble_lost'].fillna(0)

    id_vars = ['season', 'week']
    contributions_map = {
        'passer_player_id': 'fp_pass_total',
        'rusher_player_id': 'fp_rush_total',
        'receiver_player_id': 'fp_rec_total',
        'fumbled_1_player_id': 'fp_fumble_1_total',
    }
    contributions = []
    for id_col, point_col in contributions_map.items():
        tmp = df.rename(columns={id_col: 'player_id', point_col: 'points'})
        contributions.append(tmp[id_vars + ['player_id', 'points']])

    df_all_points = pd.concat(contributions, ignore_index=True)
    df_all_points.dropna(subset=['player_id'], inplace=True)

    out = df_all_points.groupby(id_vars + ['player_id'])['points'].sum().reset_index()
    return out.rename(columns={'points': 'Y_target_points'})


In [4]:
# Build modeling dataframe (df_final) from play-by-play
import nfl_data_py as nfl

# Load plays (same years as modeling notebook)
df = nfl.import_pbp_data([2022, 2023, 2024, 2025])

# Keep fantasy-relevant plays
fantasy_play_types = ['pass', 'run', 'qb_kneel']
exclude_play_types = ['no_play', 'qb_spike', 'field_goal', 'extra_point', 'punt', 'kickoff']

df_fantasy_plays = df[
    df['play_type'].isin(fantasy_play_types) & ~df['play_type'].isin(exclude_play_types)
].copy()

# Select modeling columns
mdl_cols = [
    'game_id','play_id','season','week','posteam','defteam','home_team','away_team',
    'passer_player_id','passer','rusher_player_id','rusher','receiver_player_id','receiver','fumbled_1_player_id','fumbled_2_player_id',
    'play_type','down','ydstogo','yardline_100','shotgun','no_huddle',
    'passing_yards','pass_touchdown','pass_attempt','complete_pass',
    'rushing_yards','rush_touchdown','rush_attempt',
    'receiving_yards','yards_after_catch',
    'penalty_yards','interception','fumble_lost',
    'yards_gained','epa','cpoe','td_prob'
]

df_mdl0 = df_fantasy_plays[mdl_cols].copy()

# Per-play fantasy points
df_mdl1 = calculate_fantasy_points_per_play(df_mdl0, scoring_rules)

# Weekly target Y per player
df_target_Y = calculate_weekly_fantasy_points_final(df_mdl1)

# Aggregate features per player-week
feature_agg_rules = {
    'passing_yards': 'sum',
    'rushing_yards': 'sum',
    'receiving_yards': 'sum',
    'pass_touchdown': 'sum',
    'rush_touchdown': 'sum',
    'interception': 'sum',
    'epa': 'mean',
    'cpoe': 'mean',
}

id_vars = ['season','week']
feature_cols = list(feature_agg_rules.keys())

df_select = df_mdl1[id_vars + feature_cols + ['passer_player_id','rusher_player_id','receiver_player_id']].copy()

df_X_long = pd.melt(
    df_select,
    id_vars=id_vars + feature_cols,
    value_vars=['passer_player_id','rusher_player_id','receiver_player_id'],
    var_name='role_type',
    value_name='player_id'
)
df_X_long.dropna(subset=['player_id'], inplace=True)

df_features_X = df_X_long.groupby(['season','week','player_id']).agg(feature_agg_rules).reset_index()

df_counts = df_X_long.groupby(['season','week','player_id']).size().reset_index(name='total_plays_involved')
df_features_X = pd.merge(df_features_X, df_counts, on=['season','week','player_id'], how='left')

# Merge with target and create time features
df_final = pd.merge(df_features_X, df_target_Y, on=['season','week','player_id'], how='left')
df_final['Y_target_points'] = df_final['Y_target_points'].fillna(0)

df_final.sort_values(by=['player_id','season','week'], inplace=True)

df_final['Y_lag_1'] = df_final.groupby('player_id')['Y_target_points'].shift(1)
df_final['Y_roll_avg_3'] = df_final.groupby('player_id')['Y_target_points'].transform(
    lambda x: x.shift(1).rolling(window=3, min_periods=1).mean()
)
df_final['Y_cum_avg'] = df_final.groupby('player_id')['Y_target_points'].transform(
    lambda x: x.shift(1).expanding(min_periods=1).mean()
)

for col in ['Y_lag_1','Y_roll_avg_3','Y_cum_avg']:
    df_final[col] = df_final[col].fillna(0)

print('df_final ready:', df_final.shape)
print(df_final.head())


2022 done.
2023 done.
2024 done.
2025 done.
Downcasting floats.
df_final ready: (20676, 16)
      season  week   player_id  passing_yards  rushing_yards  receiving_yards  \
0       2022     1  00-0019596          212.0           -1.0            212.0   
318     2022     2  00-0019596          190.0           -2.0            190.0   
634     2022     3  00-0019596          271.0           -1.0            271.0   
943     2022     4  00-0019596          385.0            0.0            371.0   
1254    2022     5  00-0019596          351.0           -3.0            351.0   

      pass_touchdown  rush_touchdown  interception       epa       cpoe  \
0                1.0             0.0           1.0 -0.012462   3.290235   
318              1.0             0.0           0.0 -0.123334 -13.116215   
634              1.0             0.0           0.0 -0.165223   4.887816   
943              3.0             0.0           0.0  0.179459   5.982167   
1254             1.0             0.0          

In [5]:
# Train quantile GBM models and compute predictions
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

id_cols = ['season','week','player_id']
target_col = 'Y_target_points'

X = df_final.drop(columns=id_cols + [target_col]).fillna(0)
y = df_final[target_col]

split_point = int(len(X) * 0.90)
X_train, X_test = X.iloc[:split_point], X.iloc[split_point:]
y_train, y_test = y.iloc[:split_point], y.iloc[split_point:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

quantiles = [0.10, 0.50, 0.90]
models = {}
y_preds = {}

for q in quantiles:
    model = HistGradientBoostingRegressor(
        loss='quantile', quantile=q, max_iter=500,
        learning_rate=0.05, max_depth=6, random_state=42
    )
    model.fit(X_train_scaled, y_train)
    models[q] = model
    y_preds[q] = model.predict(X_test_scaled)

import pandas as pd

df_predictions = pd.DataFrame({
    'y_test': y_test.values,
    'pred_10': y_preds[0.10],
    'pred_50': y_preds[0.50],
    'pred_90': y_preds[0.90]
}, index=y_test.index)

mae = mean_absolute_error(df_predictions['y_test'], df_predictions['pred_50'])
print(f"MAE (median): {mae:.2f}")

# Attach identifiers to the predictions (align by index)
ids_test = df_final.loc[df_predictions.index, id_cols].reset_index(drop=True)
df_preds_full = pd.concat([ids_test.reset_index(drop=True), df_predictions.reset_index(drop=True)], axis=1)
print('Predictions shape:', df_preds_full.shape)
df_preds_full.head()


MAE (median): 0.48
Predictions shape: (2068, 7)


,season,week,player_id,y_test,pred_10,pred_50,pred_90
0,2025,2,00-0039075,23.600000,16.876480,20.892123,21.982870
1,2025,3,00-0039075,17.300001,9.352937,15.747823,17.976025
2,2025,4,00-0039075,29.500000,19.641610,28.019555,30.648956
3,2025,5,00-0039075,19.500000,16.969888,19.059870,19.759117
4,2025,6,00-0039075,3.800000,3.689791,4.022108,4.057427


---
## Step 3: Evaluate Model Performance

Before using the model, let's check how accurate it is on held-out test data:
- **MAE**: How far off is our median prediction on average?
- **PICP**: How often do actual scores fall within our floor-ceiling range?
- **Floor Accuracy**: How reliable is our "safe floor" prediction?


In [6]:
# Model Evaluation Metrics (with simple explanations)
print("=" * 60)
print("MODEL EVALUATION (tested on held-out data)")
print("=" * 60)

# 1. MAE - How accurate is our median prediction?
mae = mean_absolute_error(df_predictions['y_test'], df_predictions['pred_50'])
print(f"\n1. Mean Absolute Error (MAE): {mae:.2f} points")
print("   What it means: On average, our prediction is off by this much.")
print("   Lower is better. Under 3 pts is good for fantasy.")

# 2. PICP - How well-calibrated are our uncertainty ranges?
lower = df_predictions['pred_10']
upper = df_predictions['pred_90']
covered = (df_predictions['y_test'] >= lower) & (df_predictions['y_test'] <= upper)
picp = covered.mean() * 100

print(f"\n2. Prediction Interval Coverage (PICP): {picp:.1f}%")
print("   What it means: How often actual scores fall within our floor-ceiling range.")

# 3. Pinball Loss - How good is our floor prediction?
def pinball_loss(y_true, y_pred, q):
    err = y_true - y_pred
    return np.where(err >= 0, q * err, (1 - q) * (-err)).mean()

pb_10 = pinball_loss(df_predictions['y_test'].values, df_predictions['pred_10'].values, 0.10)

# Calculate how often actual was BELOW our floor (bad - we were overconfident)
pct_below_floor = (df_predictions['y_test'] < df_predictions['pred_10']).mean() * 100

print(f"\n3. Pinball Loss (floor): {pb_10:.4f}")
print("   What it means: Measures floor prediction accuracy (lower = better).")
print(f"   - Players scored BELOW our floor {pct_below_floor:.1f}% of the time")


MODEL EVALUATION (tested on held-out data)

1. Mean Absolute Error (MAE): 0.48 points
   What it means: On average, our prediction is off by this much.
   Lower is better. Under 3 pts is good for fantasy.

2. Prediction Interval Coverage (PICP): 71.7%
   What it means: How often actual scores fall within our floor-ceiling range.

3. Pinball Loss (floor): 0.1537
   What it means: Measures floor prediction accuracy (lower = better).
   - Players scored BELOW our floor 16.3% of the time


In [7]:
# Inspect features and counts used for modeling
feature_cols_model = sorted([c for c in df_final.columns if c not in ['season','week','player_id','Y_target_points']])
print("Features used (X):", feature_cols_model)
print("Num features:", len(feature_cols_model))

num_players = df_final['player_id'].nunique()
num_player_weeks = len(df_final)
print("Unique players in df_final:", num_players)
print("Total player-weeks:", num_player_weeks)

# Optional: quick sanity on weeks per player
weeks_per_player = df_final.groupby('player_id').size()
print("Median weeks per player:", weeks_per_player.median())


Features used (X): ['Y_cum_avg', 'Y_lag_1', 'Y_roll_avg_3', 'cpoe', 'epa', 'interception', 'pass_touchdown', 'passing_yards', 'receiving_yards', 'rush_touchdown', 'rushing_yards', 'total_plays_involved']
Num features: 12
Unique players in df_final: 1027
Total player-weeks: 20676
Median weeks per player: 12.0


---
## Step 4: Prepare for Roster Predictions

Set up player ID mapping and get each player's historical performance patterns (lagged features) for matching with ESPN roster.


In [8]:
# Build player ID mapping for ESPN roster
import pandas as pd
import re
import unicodedata

# Get player ID crosswalk
ids = nfl.import_ids()
name_col_ids = 'name' if 'name' in ids.columns else 'full_name'
ids_map = ids[['gsis_id', name_col_ids]].dropna().drop_duplicates('gsis_id')
ids_map = ids_map.rename(columns={'gsis_id': 'player_id', name_col_ids: 'player_name'})

# Name normalization for matching
SUFFIXES = {'jr','sr','ii','iii','iv','v'}
def normalize_name(s: str) -> str:
    if pd.isna(s):
        return ''
    s = unicodedata.normalize('NFKD', s).encode('ASCII', 'ignore').decode('ASCII')
    s = re.sub(r"[^a-zA-Z\s]", "", s).strip().lower()
    parts = [p for p in s.split() if p and p not in SUFFIXES]
    return " ".join(parts)

ids_map['name_norm'] = ids_map['player_name'].apply(normalize_name)

# Get latest season/week info
latest_season = df_final['season'].max()
df_latest_season = df_final[df_final['season'] == latest_season].copy()
latest_week_in_data = int(df_latest_season['week'].max())
upcoming_week = latest_week_in_data + 1

print(f"Latest data: {latest_season} week {latest_week_in_data}")
print(f"Predicting for: week {upcoming_week}")

# Get each player's lagged features (from their most recent actual game)
df_player_hist = df_latest_season.sort_values(['player_id', 'week']).groupby('player_id').last().reset_index()
df_player_hist = df_player_hist.merge(ids_map[['player_id', 'player_name', 'name_norm']], on='player_id', how='left')

print(f"Players with historical data: {len(df_player_hist)}")


Latest data: 2025 week 14
Predicting for: week 15
Players with historical data: 547


---
## Step 5: Your Roster Predictions

Generate predictions for your ESPN roster using:
- **ESPN projected stats** (rushing/receiving/passing yards, TDs) for the upcoming week
- **Historical patterns** (lagged features: last week, 3-week avg, career avg)

**How to use these predictions**:
- **Need a safe floor?** Start players with high `pred_10` values
- **Need upside?** Start players with high `pred_90` values  
- **Best overall?** Sort by `pred_50` (median expected points)
- **Compare to ESPN**: `espn_proj` shows ESPN's single-point projection


In [ ]:
# Get ESPN roster with WEEKLY projected stats
# ESPN stats structure: stats[week_num] contains 'projected_points' and 'projected_breakdown'

# Find the upcoming week (highest week with projections)
sample_stats = getattr(team.roster[0], 'stats', {})
available_weeks = [w for w in sample_stats.keys() if w > 0 and 'projected_points' in sample_stats.get(w, {})]
target_week = max(available_weeks) if available_weeks else upcoming_week
print(f"Using ESPN projections for week {target_week}")

roster_data = []
for p in team.roster:
    name = getattr(p, 'name', None)
    if name and 'D/ST' not in name:
        stats = getattr(p, 'stats', {}) or {}
        
        # Get weekly projection (not season total!)
        week_data = stats.get(target_week, {})
        proj_breakdown = week_data.get('projected_breakdown', {})
        
        # Extract weekly projected stats
        espn_weekly_proj = week_data.get('projected_points', 0) or 0
        proj_rush_yds = proj_breakdown.get('rushingYards', 0) or 0
        proj_rush_td = proj_breakdown.get('rushingTouchdowns', 0) or 0
        proj_rec_yds = proj_breakdown.get('receivingYards', 0) or 0
        proj_rec_td = proj_breakdown.get('receivingTouchdowns', 0) or 0
        proj_pass_yds = proj_breakdown.get('passingYards', 0) or 0
        proj_pass_td = proj_breakdown.get('passingTouchdowns', 0) or 0
        proj_int = proj_breakdown.get('madeInterceptions', 0) or 0  # interceptions thrown
        
        roster_data.append({
            'player_name_espn': name,
            'espn_proj_pts': espn_weekly_proj,
            'passing_yards': proj_pass_yds,
            'rushing_yards': proj_rush_yds,
            'receiving_yards': proj_rec_yds,
            'pass_touchdown': proj_pass_td,
            'rush_touchdown': proj_rush_td,
            'interception': proj_int,
        })

espn_df = pd.DataFrame(roster_data)
espn_df['name_norm'] = espn_df['player_name_espn'].apply(normalize_name)

print(f"ESPN roster: {len(espn_df)} players | Using week {target_week} projections")

# Match ESPN roster to NFL data (to get lagged features)
merged = espn_df.merge(
    df_player_hist[['player_id', 'name_norm', 'Y_lag_1', 'Y_roll_avg_3', 'Y_cum_avg', 'week']],
    on='name_norm',
    how='left'
)

matched = merged[merged['player_id'].notna()].copy()
unmatched = merged[merged['player_id'].isna()]['player_name_espn'].tolist()

print(f"\nMatched {len(matched)} / {len(espn_df)} ESPN roster players to NFL data.")
if unmatched:
    print(f"Unmatched: {unmatched}")

# Build feature matrix using ESPN projections + lagged features
# Training features: passing_yards, rushing_yards, receiving_yards, pass_touchdown, 
#                   rush_touchdown, interception, epa, cpoe, total_plays_involved,
#                   Y_lag_1, Y_roll_avg_3, Y_cum_avg

# Use ESPN projections for game stats, fill epa/cpoe/plays with averages
avg_epa = df_final['epa'].mean()
avg_cpoe = df_final['cpoe'].mean()
avg_plays = df_final['total_plays_involved'].mean()

# Build X with same column order as training
feature_order = ['passing_yards', 'rushing_yards', 'receiving_yards', 'pass_touchdown', 
                 'rush_touchdown', 'interception', 'epa', 'cpoe', 'total_plays_involved',
                 'Y_lag_1', 'Y_roll_avg_3', 'Y_cum_avg']

matched['epa'] = avg_epa
matched['cpoe'] = avg_cpoe  
matched['total_plays_involved'] = avg_plays

X_roster = matched[feature_order].fillna(0)

# Scale and predict
X_roster_scaled = scaler.transform(X_roster)

matched['pred_10'] = models[0.10].predict(X_roster_scaled)
matched['pred_50'] = models[0.50].predict(X_roster_scaled)
matched['pred_90'] = models[0.90].predict(X_roster_scaled)

# Display results
print(f"\n{'='*60}")
print(f"YOUR ROSTER PREDICTIONS FOR WEEK {upcoming_week} ({latest_season} Season)")
print(f"{'='*60}")
print(f"Using: ESPN projected stats + historical performance patterns")
print(f"pred_10 = floor (90% chance to beat), pred_50 = median, pred_90 = ceiling")

output = matched[['player_name_espn', 'espn_proj_pts', 'pred_10', 'pred_50', 'pred_90']].copy()
output = output.rename(columns={'player_name_espn': 'player', 'espn_proj_pts': 'espn_proj'})
output = output.sort_values('pred_50', ascending=False).reset_index(drop=True)

# Round for readability
for col in ['pred_10', 'pred_50', 'pred_90', 'espn_proj']:
    output[col] = output[col].round(1)

# Flag players on bye or missing projections
bye_players = output[output['espn_proj'] == 0]['player'].tolist()
if bye_players:
    print(f"⚠️  Players with no ESPN projection (bye week?): {bye_players}")

display(output)


Using ESPN projections for week 14

ESPN roster: 16 players
Sample WEEKLY projections (week 14):
   player_name_espn  espn_proj_pts  rushing_yards  receiving_yards  \
0    Bijan Robinson          18.38      72.881298        40.048823   
1      Trey McBride          12.31       0.000000        63.056987   
2    Kyren Williams          12.97      67.275817        13.220511   
3       Josh Jacobs          20.34      92.707862        23.628555   
4     Davante Adams          14.49       0.000000        76.049356   
5       Tee Higgins          11.21       0.000000        60.437850   
6          DJ Moore          10.74       3.863325        56.886423   
7      Tucker Kraft           0.00       0.000000         0.000000   
8  Javonte Williams          13.25      68.989273        13.154384   
9   Hollywood Brown           2.32       0.000000        12.375237   

   passing_yards  
0            0.0  
1            0.0  
2            0.0  
3            0.0  
4            0.0  
5            0.0  

,player,espn_proj,pred_10,pred_50,pred_90
0,Justin Herbert,16.53,17.0,18.8,24.6
1,Jalen Hurts,19.82,17.5,18.4,25.0
2,Kirk Cousins,10.74,14.9,16.4,20.7
3,Josh Jacobs,20.34,14.8,15.2,14.7
4,Bijan Robinson,18.38,12.8,13.6,14.8
5,Javonte Williams,13.25,11.9,12.2,12.9
6,Kyren Williams,12.97,11.8,12.1,12.6
7,Davante Adams,14.49,4.8,7.8,8.2
8,Trey McBride,12.31,4.2,7.4,6.7
9,Tee Higgins,11.21,4.2,7.3,6.6
